# Course Scheduling - Parameter Loading
## Hackathon MVP - Week Schedule for CS & DS Programs

This notebook loads all parameters from JSON files and prepares them for the PuLP optimization model.

## 1. Import Required Libraries

In [7]:
import json
from datetime import datetime
from pprint import pprint
import pulp

# You'll need this for the optimization model later
# import pulp

## 2. Load All Parameters

Run the parameter loader script to get all data organized.

In [8]:
# Run the parameter loading script
%run load_parameters.py

Programs (P): ['CS_Y1', 'CS_Y2', 'DS_Y1', 'DS_Y2']
Courses (C): ['BCS1220', 'BCS1440', 'BCS1530', 'BCS2110', 'BCS2210', 'BCS2720', 'KEN1440', 'KEN1530', 'KEN1220', 'KEN2230', 'KEN2240', 'KEN2530']
Rooms (R): ['EDP150', 'C0.004', 'C0.008', 'C0.016', 'C0.020', 'C1.005', 'C1.015', 'C2.007']
Lecturers (L): ['Tom', 'Ottie', 'Filippe', 'Tony', 'Francessco', 'Tang', 'Marieke', 'Stefan', 'Yan', 'Ashish 10', 'Charis']

Student counts (n_p): {'CS_Y1': 220, 'CS_Y2': 165, 'DS_Y1': 180, 'DS_Y2': 218}

Courses per program (Courses_p): {'CS_Y1': ['BCS1220', 'BCS1440', 'BCS1530'], 'CS_Y2': ['BCS2110', 'BCS2210', 'BCS2210'], 'DS_Y1': ['KEN1220', 'KEN1440', 'KEN1530'], 'DS_Y2': ['KEN1220', 'KEN1440', 'KEN1530']}

Course timelines (Timeline_c): {'BCS1220': ['C', 'C', 'T'], 'BCS1440': ['C', 'C', 'T'], 'BCS1530': ['C', 'C', 'T'], 'BCS2110': ['C', 'C', 'T'], 'BCS2210': ['C', 'C', 'T'], 'BCS2720': ['C', 'C', 'T'], 'KEN1440': ['C', 'C', 'T'], 'KEN1530': ['C', 'C', 'T'], 'KEN1220': ['C', 'C', 'T'], 'KEN2230': 

## 3. Access Individual Parameters

All parameters are now available in the `params` dictionary.

In [9]:
# Access sets
programs = params['P']
courses = params['C']
rooms = params['R']
lecturers = params['L']
days = params['D']
time_slots = params['T']

print("Programs:", programs)
print("Courses:", courses)
print("Days:", days)
print("Time slots:", time_slots)

Programs: ['CS_Y1', 'CS_Y2', 'DS_Y1', 'DS_Y2']
Courses: ['BCS1220', 'BCS1440', 'BCS1530', 'BCS2110', 'BCS2210', 'BCS2720', 'KEN1440', 'KEN1530', 'KEN1220', 'KEN2230', 'KEN2240', 'KEN2530']
Days: [1, 2, 3, 4, 5]
Time slots: [1, 2, 3, 4]


In [10]:
# Access parameters
student_counts = params['n']  # n_p: students per program
program_courses = params['Courses']  # Courses_p: courses per program
course_timelines = params['Timeline']  # Timeline_c: session structure
session_indices = params['I']  # I_c: session numbers for each course
course_lecturers = params['Lecturer']  # Lecturer_c: who teaches what
room_capacities = params['Cap']  # Cap_r: room capacities
lecturer_unavailability = params['U']  # U_l,d: unavailability matrix

print("\nStudent counts:")
pprint(student_counts)


Student counts:
{'CS_Y1': 220, 'CS_Y2': 165, 'DS_Y1': 180, 'DS_Y2': 218}


## 4. USE PULP

The `params` dictionary contains everything you need:
- **Sets**: `P`, `C`, `R`, `L`, `D`, `T`
- **Parameters**: `n`, `Courses`, `Timeline`, `I`, `Lecturer`, `Cap`, `U`
- **Derived data**: `students_per_course`
- **Helper info**: `DAY_NAMES`, `TIME_SLOT_NAMES`, `WEEK_DATES`

In [11]:
# instanciating the model 

import pulp
model = pulp.LpProblem("Course_Scheduling", pulp.LpMinimize)
x = pulp.LpVariable.dicts(
    "schedule",                           # Variable name prefix
    [(c, i, r, d, t)                      # Create tuple for each combination
     for c in params['C']                 # For each course
     for i in params['I'][c]              # For each session in that course
     for r in params['R']                 # For each room
     for d in params['D']                 # For each day (1-5)
     for t in params['T']],               # For each time slot (1-4)
    cat='Binary'                          # Binary: 0 or 1
)

print(f"Created {len(x)} binary decision variables")
print(f"This represents all possible scheduling combinations!")

# ==============================================================================
# SET OBJECTIVE (FEASIBILITY ONLY)
# ==============================================================================

# We just want ANY valid schedule, so minimize 0 (dummy objective)
model += 0, "Feasibility_Only"


# ==============================================================================
# CONSTRAINT 1: Each session must be scheduled exactly once
# ==============================================================================

for c in params['C']:                    # For each course
    for i in params['I'][c]:             # For each session in that course
        model += (
            pulp.lpSum(
                x[c, i, r, d, t]         # Sum over all possible locations/times
                for r in params['R']     # All rooms
                for d in params['D']     # All days
                for t in params['T']     # All time slots
            ) == 1,                      # Must equal exactly 1
            f"Session_{c}_session{i}_scheduled_once"  # Constraint name (for debugging)
        )

# ==============================================================================
# CONSTRAINT 2: Room capacity must accommodate at least 50% of students
# ==============================================================================
#TOOD: change to lectures, and the constraint with tutorials is that there are 2 rooms for it

for c in params['C']:
    min_capacity = 0.5 * students_per_course[c]
    
    for i in params['I'][c]:
        for r in params['R']:
            if params['Cap'][r] >= min_capacity:
                # Room is big enough - add constraint
                for d in params['D']:
                    for t in params['T']:
                        model += (
                            x[c, i, r, d, t] * students_per_course[c] 
                            <= 2 * params['Cap'][r],
                            f"C2_Capacity_{c}_i{i}_r{r}_d{d}_t{t}"
                        )
            else:
                # Room too small - prevent scheduling
                for d in params['D']:
                    for t in params['T']:
                        model += (
                            x[c, i, r, d, t] == 0,
                            f"C2_TooSmall_{c}_i{i}_r{r}_d{d}_t{t}"
                        )


# ==============================================================================
# CONSTRAINT 3: A room can only host 1 session at a time
# ==============================================================================

for r in params['R']:
    for d in params['D']:
        for t in params['T']:
            model += (
                pulp.lpSum(
                    x[c, i, r, d, t]
                    for c in params['C']
                    for i in params['I'][c]
                ) <= 1,
                f"C3_NoDoubleBook_r{r}_d{d}_t{t}"
            )


# ==============================================================================
# CONSTRAINT 4: if a lecturer is unavailable that day, no lecturs of his will be made that day
# ==============================================================================

DAY_NAMES = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}

for c in params['C']:
    lecturer = params['Lecturer'][c]
    unavailable_days = params['U'].get(lecturer, [])
    
    for i in params['I'][c]:
        for d in params['D']:
            if DAY_NAMES[d] in unavailable_days:
                for t in params['T']:
                    model += (
                        pulp.lpSum(
                            x[c, i, r, d, t]
                            for r in params['R']
                        ) == 0,
                        f"C4_Unavailable_{c}_i{i}_{lecturer}_d{d}_t{t}"
                    )

# ==============================================================================
# CONSTRAINT 5: no lecturer double - booking
# ==============================================================================

for l in params['L']:
    courses_by_lecturer = [c for c in params['C'] if params['Lecturer'][c] == l]
    
    for d in params['D']:
        for t in params['T']:
            model += (
                pulp.lpSum(
                    x[c, i, r, d, t]
                    for c in courses_by_lecturer
                    for i in params['I'][c]
                    for r in params['R']
                ) <= 1,
                f"C5_NoDoubleTeach_{l}_d{d}_t{t}"
            )

# ==============================================================================
# CONSTRAINT 6: students in a program can only attend one session at a time
# ==============================================================================

for p in params['P']:
    courses_in_program = params['Courses'][p]
    
    for d in params['D']:
        for t in params['T']:
            model += (
                pulp.lpSum(
                    x[c, i, r, d, t]
                    for c in courses_in_program
                    for i in params['I'][c]
                    for r in params['R']
                ) <= 1,
                f"C6_NoStudentOverlap_{p}_d{d}_t{t}"
            )
# ==============================================================================
# CONSTRAINT 7: Sessions must happen in order
# ==============================================================================

for c in params['C']:
    for i in range(len(params['I'][c]) - 1):
        time_session_i = pulp.lpSum(
            (5 * d + t) * x[c, i, r, d, t]
            for r in params['R']
            for d in params['D']
            for t in params['T']
        )
        
        time_session_i_plus_1 = pulp.lpSum(
            (5 * d + t) * x[c, i + 1, r, d, t]
            for r in params['R']
            for d in params['D']
            for t in params['T']
        )
        
        model += (
            time_session_i + 1 <= time_session_i_plus_1,
            f"C7_Ordering_{c}_i{i}_before_i{i+1}"
        )

### SOLVE THE MODEL! ###

status = model.solve()


Created 5760 binary decision variables
This represents all possible scheduling combinations!


NameError: name 'students_per_course' is not defined

In [ ]:
if status == pulp.LpStatusOptimal:
    print("✅ Found a feasible schedule!")
    # Extract schedule from x variables
else:
    print("❌ No feasible schedule exists")